# Related_Issue_event

In [1]:
# -*- coding: utf-8 -*-
import os
import json
import re
import string
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.tokenize import MWETokenizer
from sklearn.decomposition import TruncatedSVD

import gensim
from gensim import corpora

import numpy as np
np.random.seed(222)

In [2]:
number_of_topic = 80
number_of_words_in_one_topic = 5

In [21]:
def make_topictable_per_doc(ldamodel, corpus, texts):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.
        
        if len(doc) == 0:
            topic_table = topic_table.append(pd.Series([-1]), ignore_index=True)
        
        # 모든 문서에 대해서 각각 아래를 수행
        
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num)]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [58]:
def find_release_topic(issue_num, num_topic, list_text_topic, tokenized_doc, title_doc):
    new_tokenized_doc = []
    for i in list_text_topic[issue_num]:
        if len(tokenized_doc[i]) == 0:
            continue
        new_tokenized_doc.append(tokenized_doc[i])
    new_title_doc = []
    for i in list_text_topic[issue_num]:
        if len(title_doc[i]) == 0:
            continue
        new_title_doc.append(title_doc[i])
          
    dictionary = corpora.Dictionary(new_tokenized_doc)
    
    corpus = [dictionary.doc2bow(text) for text in new_tokenized_doc]
    
    if len(corpus) != 0:
        ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = num_topic, id2word=dictionary, passes=30)
        topictable_issue = make_topictable_per_doc(ldamodel, corpus, new_tokenized_doc)
    else:
        topictable_issue = pd.DataFrame()
        for i in range(len(list_text_topic[issue_num])):
            topictable_issue = topictable_issue.append(pd.Series([0]), ignore_index=True)
    
    topictable_issue.columns = ['Issue']
    
    topictable_issue['Doc_num'] = pd.Series(list_text_topic[issue_num])
    topictable_issue[' body'] = pd.Series(new_tokenized_doc)
    topictable_issue['title'] = pd.Series(new_title_doc)

#     text_topic_list = []
#     for j in range(num_topic):
#         temp_list = [topictable_issue['title'].iloc[i] for i in range(len(topictable_issue)) if topictable_issue.loc[i, 'Issue']==j]
#         text_topic_list.append(temp_list)

#     MAX_SEQUENCE_LENGTH = 8
#     total_dict_list = []
#     for _ in range(num_topic):
#         dict_list = []
#         for _ in range(MAX_SEQUENCE_LENGTH + 1):
#             dict_list.append(dict())
#         total_dict_list.append(dict_list)

#     for k in range(num_topic):
#         topic_list = text_topic_list[k]
#         dict_list = total_dict_list[k]
#         for sequence_length in range(1, MAX_SEQUENCE_LENGTH + 1):
#         # sequence_length = 1
#             for single_topic_list in topic_list:
#                 # print(sequence_length)
#                 for i in range( len(single_topic_list) - sequence_length):
#           #         print("single_topic_list[i:i+sequence_length]")
#           #         print(single_topic_list[i:i+sequence_length])
#                     if str( single_topic_list[i:i+sequence_length] ) not in dict_list[sequence_length]:
#                         dict_list[sequence_length][str( single_topic_list[i:i+sequence_length] )] = 0
#                     dict_list[sequence_length][str( single_topic_list[i:i+sequence_length] )] += 1


#     for j in range(len(total_dict_list)):
#         print(j+1,"th Topic :")
        
#         for i in range(1, MAX_SEQUENCE_LENGTH + 1):
#             if len(total_dict_list[j][i]) == 0:
#                 continue
#             key, value = max(total_dict_list[j][i].items(), key=lambda x:x[1])
#             print(key)
#             print(value)


    return topictable_issue

In [40]:
topictable = pd.read_csv("../CSV_Data/preprocessed_data.csv")

import ast 
topictable['title'] = topictable['title'].apply(ast.literal_eval)
topictable[' body'] = topictable[' body'].apply(ast.literal_eval)

list_text_topic = [[] for i in range(number_of_topic)]
for index in range(len(topictable)):
    list_text_topic[int(topictable.loc[index, 'Issue'])].append(index)
    

In [59]:
eventtable = []
for i in range(number_of_topic):
    print(i, "th Issue")
    if len(list_text_topic[i]) == 0:
        eventtable.append([])
        continue
    eventtable.append(find_release_topic(i, 6, list_text_topic, topictable[' body'], topictable['title']))

0 th Issue
1 th Issue
2 th Issue
3 th Issue
4 th Issue
5 th Issue
6 th Issue
7 th Issue
8 th Issue
9 th Issue
10 th Issue
11 th Issue
12 th Issue
13 th Issue
14 th Issue
15 th Issue
16 th Issue
17 th Issue
18 th Issue
19 th Issue
20 th Issue
21 th Issue
22 th Issue
23 th Issue
24 th Issue
25 th Issue
26 th Issue
27 th Issue
28 th Issue
29 th Issue
30 th Issue
31 th Issue
32 th Issue
33 th Issue
34 th Issue
35 th Issue
36 th Issue
37 th Issue
38 th Issue
39 th Issue
40 th Issue
41 th Issue
42 th Issue
43 th Issue
44 th Issue
45 th Issue
46 th Issue
47 th Issue
48 th Issue
49 th Issue
50 th Issue
51 th Issue
52 th Issue
53 th Issue
54 th Issue
55 th Issue
56 th Issue
57 th Issue
58 th Issue
59 th Issue
60 th Issue
61 th Issue
62 th Issue
63 th Issue
64 th Issue
65 th Issue
66 th Issue
67 th Issue
68 th Issue
69 th Issue
70 th Issue
71 th Issue
72 th Issue
73 th Issue
74 th Issue
75 th Issue
76 th Issue
77 th Issue
78 th Issue
79 th Issue


In [60]:
event_index = pd.DataFrame()

for i in range(number_of_topic):
    if len(eventtable[i]) == 0:
        continue
    event_index = event_index.append(eventtable[i])

event_index.sort_values('Doc_num', inplace = True)

event_index = event_index.reset_index()

In [66]:
csv_data = pd.read_csv("../CSV_Data/ner_tagged.csv")

csv_data['Issue'] = topictable['Issue']
csv_data['Event'] = event_index['Issue']


csv_data.to_csv("../CSV_Data/related_issue_event.csv")
csv_data

,Unnamed: 0,title,author,time,description,body,section,index,Geographical Entity,Organization,Person,Geopolitical Entity,Time indicator,Artifact,Event,Natural Phenomenon,Issue
0,0,A snapshot of multiculturalism in South Korea,Lee Sun-young,2018-01-01 17:07:00,With birthrates persistently low and the senio...,With birthrates persistently low and the senio...,Social affairs,0,"south korea,south korea,china,statistics korea...",NaN,"birthrates persistently,549 ,000","korean,korean,korean,korean,chinese","2018 onward,2016,1990s,2008 , 11,7,7,2016,2016...",NaN,2.0,NaN,3.0
1,1,[Weekender] Korea’s dynamic 2017,Choi He-suk,2018-01-01 13:22:00,From North Korea’s nuclear weapons program nea...,From North Korea’s nuclear weapons program nea...,Social affairs,1,"north korea ’s,south korea,grace,north korea,n...","park ’s,“accumulated ills ,”,pyongyang,“red li...","president park geun-hye ’s,jae-in,(yonhap)moon...","korean,korean","2016,march,may 9,2008,sept,may,100th day,augus...",NaN,2.0,NaN,36.0
2,2,People's Party members support Ahn's push for ...,Yonhap,2017-12-31 16:18:00,The leader of the center-left People's Party g...,The leader of the center-left People's Party g...,Politics,2,"korea,299-member national assembly","ahn,nonconfidence for ahn,bareun party has11","ahn cheol-soo,lee dong-seop",NaN,"sunday,wednesday,saturday,sunday,dec,june,june...",NaN,2.0,NaN,31.0
3,3,[Newsmaker] Panamanian vessel probed over susp...,Yonhap,2017-12-31 14:55:00,PYEONGTAEK -- South Korea has seized and insp...,PYEONGTAEK -- South Korea has seized and insp...,North Korea,3,"pyeongtaek,south korea,north korea,u.n.,pyeong...","pyeongtaek harbor (yonhap)it,foreign ministry,...","maritimeauthorities,,100-ton koti,unsc,2375","chinese,korean,korean","dec,friday,oct,yeosu,nov,24,september",NaN,3.0,NaN,39.0
4,4,Hong Kong ship crew questioned in S. Korea for...,AFP,2017-12-30 15:44:00,The crew of a Hong Kong-registered ship have b...,The crew of a Hong Kong-registered ship have b...,North Korea,4,"hong kong-registered,south korea,yeosu,korea,y...","breaching un,lighthouse winmore,afp,un securit...","(afp-yonhap)the lighthouse winmore,,000 tonnes...","korean,taiwanese,korean,korean,chinese,taiwane...","november,saturday,november 24,november 24,yeos...",NaN,3.0,NaN,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23764,23764,N. Korean leader's speech arouses cautious opt...,KH디지털2,2015-01-01 13:36:00,North Korean leader Kim Jong-un's New Year's D...,North Korean leader Kim Jong-un's New Year's D...,North Korea,23764,"north korean,2015,korea,korea,japan,north kore...","seoul national university institute,university...","kim jong-un,kim,kim,kim,kim,kim,chang yong-seo...","korean president park geun-hye,korean,korean","new,thursday,70th,1910,minister-level,january,...",NaN,0.0,NaN,59.0
23765,23765,N. Korean leader open to inter-Korean summit t...,KH디지털2,2015-01-01 10:05:00,North Korean leader Kim Jong-un said Thursday ...,North Korean leader Kim Jong-un said Thursday ...,North Korea,23765,"seoul,mount kumgang,reunion,south korea,japan,...",pyongyang,"kim jong-un,(yonhap)kim,kim echoed park,kim,ki...","korean,korean president park geun-hye,korean,k...","thursday,new,koreas,january,inter-korean ties,...",NaN,0.0,NaN,59.0
23766,23766,Ex-U.S. envoy calls for clearer communication ...,KH디지털2,2015-01-01 09:27:00,The United States should make its thoughts on ...,The United States should make its thoughts on ...,North Korea,23766,"united states,north korea,china,beijing,north,...","us,us,us,sino-american cooperation","christopher hill,hill,'no surprises,hill","korean,american,chinese,korean,chinese,chinese...",wednesday,NaN,3.0,NaN,59.0
23767,23767,U.S. imposes sanctions on N. Korean firm,KH디지털2,2015-01-01 09:25:00,The United States has imposed sanctions on a N...,The United States has imposed sanctions on a N...,North Korea,23767,"united states,china,iran,iran,north korea,syri...","state department,state department",NaN,"korean,korean,korean,russ